In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering, KMeans, DBSCAN
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import MinMaxScaler

plt.rc('font', family='NanumBarunGothic')

from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
pd.set_option('display.max_colwidth', None)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
table_12 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/최종프/각 전처리 파일 모음/merge2_onlyC.csv")
table_34 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/최종프/각 전처리 파일 모음/v2_cont_merge_ver1_(not_drop).csv')
table_34_drop = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/최종프/각 전처리 파일 모음/v2_cont_merge_ver2_(drop).csv')
table_567 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/최종프/각 전처리 파일 모음/pmntdtl_pol (1).csv')


In [3]:
display(table_12.head(), table_12.shape)
display(table_34.head(),table_34.shape )
display(table_34_drop.head(),table_34_drop.shape)
display(table_567.head(), table_567.shape)

,JOIN_SN,POL_SN,CT_RLPS_GBN_CD,GENDER,AGE,AGE_GROUP
0,1486482,1771111,C,1,34,3
1,15110,2197182,C,2,25,2
2,993239,2471577,C,2,68,6
3,745358,566902,C,2,30,3
4,56423,254040,C,2,28,2


(129442, 6)

,POL_SN,reduce_GBN_CD,reduce_STAT_CD,CT_CHNL_CD,CT_PY_AMT,CT_PY_CYCLE_CD,CT_PY_PD,mean_AMT,GBN_CD_1,GBN_CD_2,GBN_CD_3,GBN_CD_4
0,37,6,4,1,3,1,0,3.0,0.0,0.0,0.0,0.0
1,43,1,2,4,5000,1,0,5000.0,0.0,0.0,0.0,0.0
2,66,3,2,4,98,3,1,98.0,0.0,0.0,0.0,0.0
3,87,7,1,1,1300,2,2,1300.0,0.0,0.0,0.0,0.0
4,111,1,1,3,60,2,25,60.0,1.0,1.0,1.0,0.0


(225663, 12)

,POL_SN,reduce_GBN_CD,reduce_STAT_CD,CT_CHNL_CD,CT_PY_AMT,CT_PY_CYCLE_CD,CT_PY_PD,mean_AMT,GBN_CD_1,GBN_CD_2,GBN_CD_3,GBN_CD_4
0,111,1,1,3,60,2,25,60.0,1,1,1,0
1,179,1,1,1,330,2,20,330.0,1,1,1,0
2,311,3,1,4,120,3,1,120.0,1,1,1,0
3,478,4,1,1,110,2,20,110.0,1,1,1,1
4,722,3,1,1,28,2,30,28.0,1,1,1,1


(110845, 12)

,JOIN_SN,POL_SN,INS_GBN_CD,총지급금액,총지급 건수,기타,배책,사망/후유,수술/입원,실손,자동차,진단/통원,화재/재물
0,8,6261020,e,390,1,1,0,0,0,0,0,0,0
1,98,8528823,d,63,2,0,0,0,0,2,0,0,0
2,514,3598146,b,467,2,0,0,0,0,2,0,0,0
3,514,5594473,i,270,3,0,0,0,0,0,3,0,0
4,514,5627695,b,48,1,0,0,0,1,0,0,0,0


(23368, 13)

In [4]:
#차주번호에는 없는 계약자정보드이 십만건있음
display(table_34[~table_34['POL_SN'].isin(table_12['POL_SN'])])

display(table_34_drop[~table_34_drop['POL_SN'].isin(table_12['POL_SN'])])

,POL_SN,reduce_GBN_CD,reduce_STAT_CD,CT_CHNL_CD,CT_PY_AMT,CT_PY_CYCLE_CD,CT_PY_PD,mean_AMT,GBN_CD_1,GBN_CD_2,GBN_CD_3,GBN_CD_4
2,66,3,2,4,98,3,1,98.000000,0.0,0.0,0.0,0.0
10,255,7,2,99,88,2,5,88.000000,0.0,0.0,0.0,0.0
11,265,3,2,4,63,1,0,63.000000,0.0,0.0,0.0,0.0
13,311,3,1,4,120,3,1,120.000000,1.0,1.0,1.0,0.0
15,382,3,2,4,87,3,1,87.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
225654,9996550,4,1,1,23,2,10,29.333333,1.0,1.0,1.0,1.0
225657,9996733,4,1,1,14,2,10,14.000000,1.0,1.0,1.0,1.0
225659,9997240,6,4,3,24,1,0,24.000000,1.0,0.0,0.0,0.0
225661,9997320,4,2,1,81,2,30,81.000000,1.0,1.0,1.0,1.0


,POL_SN,reduce_GBN_CD,reduce_STAT_CD,CT_CHNL_CD,CT_PY_AMT,CT_PY_CYCLE_CD,CT_PY_PD,mean_AMT,GBN_CD_1,GBN_CD_2,GBN_CD_3,GBN_CD_4
2,311,3,1,4,120,3,1,120.000000,1,1,1,0
3,478,4,1,1,110,2,20,110.000000,1,1,1,1
5,876,7,1,1,45,2,15,45.000000,1,0,0,0
6,882,4,1,99,80,2,20,80.000000,1,1,1,1
9,1308,4,1,3,30,2,30,30.000000,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
110837,9996422,5,2,3,30,2,20,30.000000,1,0,0,1
110839,9996550,4,1,1,23,2,10,29.333333,1,1,1,1
110841,9996733,4,1,1,14,2,10,14.000000,1,1,1,1
110843,9997240,6,4,3,24,1,0,24.000000,1,0,0,0


In [6]:
all_merge_d = pd.merge(table_12,table_34_drop, how='inner',on='POL_SN')
all_merge_d = pd.merge(all_merge_d, table_567, how ='left', on = ['JOIN_SN'	,'POL_SN'])
all_merge_d

,JOIN_SN,POL_SN,CT_RLPS_GBN_CD,GENDER,AGE,AGE_GROUP,reduce_GBN_CD,reduce_STAT_CD,CT_CHNL_CD,CT_PY_AMT,...,총지급금액,총지급 건수,기타,배책,사망/후유,수술/입원,실손,자동차,진단/통원,화재/재물
0,15110,2197182,C,2,25,2,7,1,4,5000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,56423,254040,C,2,28,2,1,1,1,170,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1659127,5420542,C,2,46,4,7,1,5,180,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,817293,5884271,C,1,41,4,6,4,7,110,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,798115,6197631,C,1,54,5,3,1,1,50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58424,1892963,8605817,C,1,36,3,6,4,7,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58425,1898196,5285028,C,1,25,2,6,2,99,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58426,1942749,6650657,C,1,24,2,6,4,7,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58427,1946706,8536828,C,2,52,5,6,4,7,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
all_merge_d[all_merge_d.duplicated(subset='POL_SN', keep= False)]

,JOIN_SN,POL_SN,CT_RLPS_GBN_CD,GENDER,AGE,AGE_GROUP,reduce_GBN_CD,reduce_STAT_CD,CT_CHNL_CD,CT_PY_AMT,...,총지급금액,총지급 건수,기타,배책,사망/후유,수술/입원,실손,자동차,진단/통원,화재/재물
11669,263297,9108814,C,2,37,3,4,1,1,73,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11670,263297,9108814,C,2,37,3,4,1,1,73,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
#중복컬럼 삭제
all_merge_d.drop_duplicates(subset=['POL_SN'], keep=False, inplace = True)

In [8]:
all_merge_d.fillna(0, inplace = True)

In [9]:
all_merge_d.T

,0,1,2,3,4,5,6,7,8,9,...,58419,58420,58421,58422,58423,58424,58425,58426,58427,58428
JOIN_SN,15110,56423,1659127,817293,798115,1875818,1544628,111496,797509,657563,...,1870705,1872115,1880016,1880016,1885935,1892963,1898196,1942749,1946706,1957937
POL_SN,2197182,254040,5420542,5884271,6197631,6530014,7514669,9524202,9351218,5707253,...,8421299,6622451,6323651,7525715,9350920,8605817,5285028,6650657,8536828,2406758
CT_RLPS_GBN_CD,C,C,C,C,C,C,C,C,C,C,...,C,C,C,C,C,C,C,C,C,C
GENDER,2,2,2,1,1,1,1,1,1,1,...,1,2,1,1,1,1,1,1,2,2
AGE,25,28,46,41,54,62,25,54,35,40,...,26,44,80,80,28,36,25,24,52,23
AGE_GROUP,2,2,4,4,5,6,2,5,3,4,...,2,4,8,8,2,3,2,2,5,2
reduce_GBN_CD,7,1,7,6,3,5,3,5,3,2,...,6,6,6,6,6,6,6,6,6,6
reduce_STAT_CD,1,1,1,4,1,1,1,2,1,1,...,4,4,4,4,4,4,2,4,4,4
CT_CHNL_CD,4,1,5,7,1,1,3,6,2,6,...,7,7,7,7,2,7,99,7,7,1
CT_PY_AMT,5000,170,180,110,50,15,140,280,60,18,...,3,17,8,11,170,10,1,6,8,4


In [10]:
#컬럼 자료형 변환
change_col = ['총지급금액','총지급건수','기타','배책','사망/후유','수술/입원','실손','자동차','진단/통원','화재/재물']
to_int = all_merge_d.columns[-10:]
all_merge_d[to_int] = all_merge_d[to_int].astype(int)
all_merge_d[to_int].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58429 entries, 0 to 58428
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   총지급금액   58429 non-null  int64
 1   총지급 건수  58429 non-null  int64
 2   기타      58429 non-null  int64
 3   배책      58429 non-null  int64
 4   사망/후유   58429 non-null  int64
 5   수술/입원   58429 non-null  int64
 6   실손      58429 non-null  int64
 7   자동차     58429 non-null  int64
 8   진단/통원   58429 non-null  int64
 9   화재/재물   58429 non-null  int64
dtypes: int64(10)
memory usage: 4.5 MB


In [12]:
all_merge_d = all_merge_d.reset_index()
all_merge_d

,index,JOIN_SN,POL_SN,CT_RLPS_GBN_CD,GENDER,AGE,AGE_GROUP,reduce_GBN_CD,reduce_STAT_CD,CT_CHNL_CD,...,총지급금액,총지급 건수,기타,배책,사망/후유,수술/입원,실손,자동차,진단/통원,화재/재물
0,0,15110,2197182,C,2,25,2,7,1,4,...,0,0,0,0,0,0,0,0,0,0
1,1,56423,254040,C,2,28,2,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,2,1659127,5420542,C,2,46,4,7,1,5,...,0,0,0,0,0,0,0,0,0,0
3,3,817293,5884271,C,1,41,4,6,4,7,...,0,0,0,0,0,0,0,0,0,0
4,4,798115,6197631,C,1,54,5,3,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58424,58424,1892963,8605817,C,1,36,3,6,4,7,...,0,0,0,0,0,0,0,0,0,0
58425,58425,1898196,5285028,C,1,25,2,6,2,99,...,0,0,0,0,0,0,0,0,0,0
58426,58426,1942749,6650657,C,1,24,2,6,4,7,...,0,0,0,0,0,0,0,0,0,0
58427,58427,1946706,8536828,C,2,52,5,6,4,7,...,0,0,0,0,0,0,0,0,0,0


In [30]:
#all_merge_d.to_csv('final.csv', index=False)

key값 : JOIN_SN    
예비키 : POL_SN   
연속형 변수 -   , CT_PY_PD(단위 : 년), mean_AMT, 총지급금액, 총지급 건수, '기타','배책', '사망/후유', '수술/입원', '실손', '자동차', '진단/통원', '화재/재물',
범주형 변수-reduce_GBN_CD	, reduce_STAT_CD, CT_CHNL_CD	, CT_PY_CYCLE_CD, mgg_GBN_CD,
삭제컬럼 - CT_RLPS_GBN_CD, AGE, CT_PY_AMT
*age_group추가 축소시 생애주기?(미혼, 기혼, 자녀키우는 시기, 떠나고 노후준비하는 시기...)

In [14]:
all_merge_d.columns

Index(['index', 'JOIN_SN', 'POL_SN', 'CT_RLPS_GBN_CD', 'GENDER', 'AGE',
       'AGE_GROUP', 'reduce_GBN_CD', 'reduce_STAT_CD', 'CT_CHNL_CD',
       'CT_PY_AMT', 'CT_PY_CYCLE_CD', 'CT_PY_PD', 'mean_AMT', 'GBN_CD_1',
       'GBN_CD_2', 'GBN_CD_3', 'GBN_CD_4', 'INS_GBN_CD', '총지급금액', '총지급 건수',
       '기타', '배책', '사망/후유', '수술/입원', '실손', '자동차', '진단/통원', '화재/재물'],
      dtype='object')

In [ ]:
#1차 불필요한 컬럼 선별
#CT_RLPS_GBN_CD, 'AGE', 'CT_PY_AMT, INS_GBN_CD'

#컬럼명 변경
# reduce_GBN_CD', 'reduce_STAT_CD''mean_AMT', 'GBN_CD_1','GBN_CD_2', 'GBN_CD_3', 'GBN_CD_4',

In [20]:
#그대로 사용할 컬럼만 추출
df1 = all_merge_d[['CT_PY_PD','mean_AMT','GBN_CD_1','GBN_CD_2', 'GBN_CD_3', 'GBN_CD_4','총지급금액', '총지급 건수',
       '기타', '배책', '사망/후유', '수술/입원', '실손', '자동차', '진단/통원', '화재/재물']]
df1.rename(columns = {"CT_PY_PD": '납입기간', "mean_AMT": "평균납입금액",'GBN_CD_1':'보험코드1', 'GBN_CD_2':'보험코드2',
                      'GBN_CD_3' : '보험코드3','GBN_CD_4':'보험코드4'}, inplace=True)
#스케일링 필요한 컬럼
# '납입기간','평균납입금액,'총지급금액', '총지급 건수','기타', '배책', '사망/후유', '수술/입원', '실손', '자동차', '진단/통원', '화재/재물'

<ipython-input-20-53eda5229028>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.rename(columns = {"CT_PY_PD": '납입기간', "mean_AMT": "평균납입금액",'GBN_CD_1':'보험코드1', 'GBN_CD_2':'보험코드2',


In [21]:
#범주형 컬럼 원핫인코딩
df1 = pd.concat([pd.get_dummies(all_merge_d['GENDER'], prefix="성별", dtype=int),
                 pd.get_dummies(all_merge_d['AGE_GROUP'], prefix="연령대", dtype=int),
                 pd.get_dummies(all_merge_d['reduce_GBN_CD'], prefix="보험종류", dtype=int),
                 pd.get_dummies(all_merge_d['reduce_STAT_CD'], prefix="보험상태", dtype=int),
                 pd.get_dummies(all_merge_d['CT_CHNL_CD'], prefix="채널", dtype=int),
                 pd.get_dummies(all_merge_d['CT_PY_CYCLE_CD'], prefix="납입주기", dtype=int),df1], axis=1)
df1.T

,0,1,2,3,4,5,6,7,8,9,...,58419,58420,58421,58422,58423,58424,58425,58426,58427,58428
성별_1,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
성별_2,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
연령대_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
연령대_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
연령대_2,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0
연령대_3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
연령대_4,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
연령대_5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
연령대_6,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
연령대_7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
# merge_dd_dumi.to_csv('fi_inco.csv', index=False, encoding='utf-8-sig')

In [23]:
print(df1.shape)
print(df1.columns)

(58429, 52)
Index(['성별_1', '성별_2', '연령대_0', '연령대_1', '연령대_2', '연령대_3', '연령대_4', '연령대_5',
       '연령대_6', '연령대_7', '연령대_8', '보험종류_1', '보험종류_2', '보험종류_3', '보험종류_4',
       '보험종류_5', '보험종류_6', '보험종류_7', '보험종류_8', '보험종류_9', '보험상태_1', '보험상태_2',
       '보험상태_3', '보험상태_4', '채널_1', '채널_2', '채널_3', '채널_4', '채널_5', '채널_6',
       '채널_7', '채널_99', '납입주기_1', '납입주기_2', '납입주기_3', '납입주기_99', '납입기간',
       '평균납입금액', '보험코드1', '보험코드2', '보험코드3', '보험코드4', '총지급금액', '총지급 건수', '기타',
       '배책', '사망/후유', '수술/입원', '실손', '자동차', '진단/통원', '화재/재물'],
      dtype='object')


In [24]:
#스케일러로 minmax사용 이유 : 기존데이터의 분포를 유지하기 위해
#변환컬럼
#'납입기간','평균납입금액,'총지급금액', '총지급 건수','기타', '배책', '사망/후유', '수술/입원', '실손', '자동차', '진단/통원', '화재/재물'
scaler = MinMaxScaler()
mm = pd.DataFrame(scaler.fit_transform(df1[['납입기간','평균납입금액','총지급금액', '총지급 건수','기타', '배책', '사망/후유', '수술/입원', '실손', '자동차', '진단/통원', '화재/재물']]),
                  columns=('납입기간mm','평균납입금액mm','총지급금액mm', '총지급 건수mm','기타mm', '배책mm', '사망/후유mm', '수술/입원mm', '실손mm', '자동차mm', '진단/통원mm', '화재/재물mm'))
mm

,납입기간mm,평균납입금액mm,총지급금액mm,총지급 건수mm,기타mm,배책mm,사망/후유mm,수술/입원mm,실손mm,자동차mm,진단/통원mm,화재/재물mm
0,0.000000,0.083318,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.050505,0.002817,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.030303,0.002983,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.001817,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.202020,0.000817,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
58424,0.000000,0.000150,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58425,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58426,0.000000,0.000083,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58427,0.000000,0.000117,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
df1 = pd.concat([df1,mm], axis =1 )
df1

,성별_1,성별_2,연령대_0,연령대_1,연령대_2,연령대_3,연령대_4,연령대_5,연령대_6,연령대_7,...,총지급금액mm,총지급 건수mm,기타mm,배책mm,사망/후유mm,수술/입원mm,실손mm,자동차mm,진단/통원mm,화재/재물mm
0,0,1,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,1,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,1,0,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,0,0,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,0,0,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58424,1,0,0,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58425,1,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58426,1,0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58427,0,1,0,0,0,0,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
df1.columns

Index(['성별_1', '성별_2', '연령대_0', '연령대_1', '연령대_2', '연령대_3', '연령대_4', '연령대_5',
       '연령대_6', '연령대_7', '연령대_8', '보험종류_1', '보험종류_2', '보험종류_3', '보험종류_4',
       '보험종류_5', '보험종류_6', '보험종류_7', '보험종류_8', '보험종류_9', '보험상태_1', '보험상태_2',
       '보험상태_3', '보험상태_4', '채널_1', '채널_2', '채널_3', '채널_4', '채널_5', '채널_6',
       '채널_7', '채널_99', '납입주기_1', '납입주기_2', '납입주기_3', '납입주기_99', '납입기간',
       '평균납입금액', '보험코드1', '보험코드2', '보험코드3', '보험코드4', '총지급금액', '총지급 건수', '기타',
       '배책', '사망/후유', '수술/입원', '실손', '자동차', '진단/통원', '화재/재물', '납입기간mm',
       '평균납입금액mm', '총지급금액mm', '총지급 건수mm', '기타mm', '배책mm', '사망/후유mm', '수술/입원mm',
       '실손mm', '자동차mm', '진단/통원mm', '화재/재물mm'],
      dtype='object')

In [28]:
#2차 불필요 컬럼
drop =['납입기간','평균납입금액', '보험코드1', '보험코드2', '보험코드3', '보험코드4', '총지급금액', '총지급 건수', '기타',
       '배책', '사망/후유', '수술/입원', '실손', '자동차', '진단/통원', '화재/재물']
df1.drop(columns =drop, axis=1, inplace=True)
df1.columns

Index(['성별_1', '성별_2', '연령대_0', '연령대_1', '연령대_2', '연령대_3', '연령대_4', '연령대_5',
       '연령대_6', '연령대_7', '연령대_8', '보험종류_1', '보험종류_2', '보험종류_3', '보험종류_4',
       '보험종류_5', '보험종류_6', '보험종류_7', '보험종류_8', '보험종류_9', '보험상태_1', '보험상태_2',
       '보험상태_3', '보험상태_4', '채널_1', '채널_2', '채널_3', '채널_4', '채널_5', '채널_6',
       '채널_7', '채널_99', '납입주기_1', '납입주기_2', '납입주기_3', '납입주기_99', '납입기간mm',
       '평균납입금액mm', '총지급금액mm', '총지급 건수mm', '기타mm', '배책mm', '사망/후유mm', '수술/입원mm',
       '실손mm', '자동차mm', '진단/통원mm', '화재/재물mm'],
      dtype='object')

### cos 유사도 방법 시도

In [69]:
!pip install soyclustering

In [70]:
from soyclustering import SphericalKMeans

spherical_kmeans = SphericalKMeans(
    n_clusters=100,
    max_iter=1000,
    verbose=1,
    init='similar_cut',
    random_state=9
)
s_labels = spherical_kmeans.fit_predict(df1)

# 틀려먹었다 일단 k-means 돌려본다

In [31]:
#K-means++ 로 초기값 지정
kmeans_pp = KMeans(n_clusters=1000, random_state=42).fit(df1)

df_centroids_pp = pd.DataFrame(kmeans_pp.cluster_centers_, columns=df1.columns)

from collections import Counter
c = Counter()
c.update(kmeans_pp.labels_)
label_counts = [c[x] for x in df_centroids_pp.index]
df_centroids_pp['size'] = label_counts

df_centroids_pp


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,성별_1,성별_2,연령대_0,연령대_1,연령대_2,연령대_3,연령대_4,연령대_5,연령대_6,연령대_7,...,총지급 건수mm,기타mm,배책mm,사망/후유mm,수술/입원mm,실손mm,자동차mm,진단/통원mm,화재/재물mm,size
0,1.000000e+00,-1.110223e-16,4.545455e-02,4.545455e-02,1.387779e-17,1.818182e-01,2.775558e-17,0.000000e+00,7.272727e-01,-1.387779e-17,...,0.000000,-2.168404e-19,1.084202e-19,1.084202e-19,2.168404e-19,0.000000,3.388132e-21,-4.336809e-19,-5.421011e-20,22
1,1.000000e+00,-8.881784e-16,2.688821e-17,1.040834e-17,6.383782e-16,8.326673e-17,2.775558e-16,1.000000e+00,6.661338e-16,3.469447e-16,...,0.008730,8.068854e-04,-1.192622e-18,1.084202e-18,3.491124e-03,0.010837,-9.147956e-20,4.191321e-03,-4.336809e-19,676
2,-7.771561e-16,1.000000e+00,5.637851e-18,-6.245005e-17,1.249001e-16,1.000000e+00,5.551115e-16,-6.938894e-16,-4.440892e-16,2.775558e-16,...,0.012904,4.316912e-03,5.586592e-03,9.757820e-19,2.346369e-03,0.016881,3.388132e-20,1.862197e-03,5.586592e-03,358
3,2.053913e-15,1.000000e+00,3.165870e-17,5.898060e-17,7.632783e-16,1.443290e-15,-9.714451e-16,1.000000e+00,3.885781e-16,3.608225e-16,...,0.005856,2.141557e-04,5.889282e-04,5.889282e-04,3.392226e-03,0.006401,-1.490778e-19,5.300353e-03,-4.878910e-19,849
4,1.000000e+00,3.330669e-16,4.336809e-19,-2.775558e-17,-1.387779e-17,-3.885781e-16,1.000000e+00,-6.383782e-16,-1.942890e-16,2.359224e-16,...,0.000593,1.913876e-03,1.951564e-18,8.673617e-19,4.210526e-04,0.000076,2.710505e-20,1.169591e-03,-3.794708e-19,285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1.000000e+00,-1.110223e-16,0.000000e+00,0.000000e+00,0.000000e+00,-2.775558e-17,-2.775558e-17,1.000000e+00,0.000000e+00,-1.387779e-17,...,0.002817,-2.168404e-19,-1.084202e-19,0.000000e+00,-2.168404e-19,0.004348,0.000000e+00,0.000000e+00,-5.421011e-20,10
996,5.551115e-17,1.000000e+00,1.734723e-18,9.523810e-02,2.775558e-17,3.809524e-01,3.809524e-01,0.000000e+00,1.428571e-01,-1.387779e-17,...,0.000000,0.000000e+00,1.084202e-19,1.084202e-19,4.336809e-19,0.000000,6.776264e-21,0.000000e+00,-5.421011e-20,21
997,1.000000e+00,-1.110223e-16,1.301043e-18,7.222222e-01,2.775558e-17,2.775558e-17,2.775558e-17,0.000000e+00,0.000000e+00,2.777778e-01,...,0.000000,0.000000e+00,1.084202e-19,1.084202e-19,4.336809e-19,0.000000,3.388132e-21,0.000000e+00,-5.421011e-20,18
998,1.000000e+00,1.110223e-16,0.000000e+00,1.428571e-01,1.387779e-17,0.000000e+00,0.000000e+00,0.000000e+00,8.571429e-01,1.387779e-17,...,0.006036,2.597403e-02,0.000000e+00,0.000000e+00,2.168404e-19,0.003106,0.000000e+00,4.336809e-19,5.421011e-20,7


In [32]:
#Centroid 시각화를 통해 각 군집의 특성 파악
import plotly.graph_objects as go

def plot_radar_from_centroid(df_centroids):
  df_centroids = df_centroids.drop(['size'], axis = 1)
  fig = go.Figure()
  categories = df_centroids.columns
  for row in df_centroids.iterrows():
    fig.add_trace(go.Scatterpolar(
        r=row[1].tolist(),
        theta=categories,
        fill='toself',
        name='cluster {}'.format(row[0])
    ))

  fig.update_layout(
      autosize=False,
      width=1000,
      height=800,
  )
  fig.show()

plot_radar_from_centroid(df_centroids_pp)

In [33]:
from sklearn.metrics import silhouette_score, silhouette_samples

silhouette_avg = silhouette_score(df1, kmeans_pp.labels_)
print(silhouette_avg)

0.8484196652634592


In [35]:
#K값에 따른 Silhouette 계수 비교
for k in range(100, 1000, 100):
    kmeans = KMeans(n_clusters=k, random_state=0).fit(df1)
    silhouette_avg = silhouette_score(df1, kmeans.labels_)
    print("k = {} -> silhouette index {}".format(k, silhouette_avg))

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



k = 100 -> silhouette index 0.326660864303987


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



k = 200 -> silhouette index 0.5037488658610946


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



...

k = 700 -> silhouette index 0.7961835882725301


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



k = 800 -> silhouette index 0.8192492896322541


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



k = 900 -> silhouette index 0.8323513315741233


In [ ]:
#K값에 따른 Silhouette 계수 비교
for k in range(1100, 2000, 100):
    kmeans = KMeans(n_clusters=k, random_state=0).fit(df1)
    silhouette_avg = silhouette_score(df1, kmeans.labels_)
    print("k = {} -> silhouette index {}".format(k, silhouette_avg))

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



k = 1100 -> silhouette index 0.859290764746335


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



k = 1200 -> silhouette index 0.866681422556226


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

